In [1]:
import csv
from glob import glob
from pybedtools import BedTool
from collections import defaultdict
from collections import Counter
from ete3 import Tree
from ete3 import PhyloTree
from itertools import chain
import itertools
import pandas as pd  
from numpy import nan
from copy import deepcopy

/Users/fmarletaz/miniconda3/envs/py39/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/fmarletaz/miniconda3/envs/py39/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def genePos(gfile):
    genePos,geneName={},{}
    for rc in csv.reader(open(gfile),delimiter='\t'):
        if rc[0].startswith('Gene'): continue
        genePos[rc[0]]=(rc[1],int(rc[2]),int(rc[3]))
        if len(rc)>=5:
            geneName[rc[0]]=rc[4]
        else:
            geneName[rc[0]]='NA'
    return genePos,geneName

In [4]:
def getAnc(nd,sptr):
    tsp=list(set([lf.species for lf in nd])) 
    tanc=sptr.get_common_ancestor(tsp).name if len(tsp)>1 else tsp[0]
    return tanc

In [5]:
def decoTree(t,outtax=['Acapla','Strpur','Sackow','Ptyfla']):
    for l in t:
        sp,gid=l.name.split('_',1)
        gid=gid.split('.')[0]
        l.add_features(species=sp,gid=gid)
    #first, reset rooting with mid-point rooting
    R = t.get_midpoint_outgroup()
    t.set_outgroup(R)
    #then root with deuterostome ancestor
    sot=[l for l in t if l.species in outtax] 
    if len(sot)>0:
        anc=t.get_common_ancestor(sot)
        if not anc==t:
            t.set_outgroup(anc)
    return t

In [6]:
def checkDup(t,spPos):
    # determine most frequent CLG in case several amphioxus genes have distinct clgs
    BflTax=[l for l in t if l.species=='Braflo']        
    clgs=[BflN[l.gid] for l in BflTax]
    clgCt=Counter(clgs)
    #print(clgCt)
    mclg=clgCt.most_common(1)[0][0]
    anDup=defaultdict(list)
    for l in t:
        if l.species in spPos:
            if not l.gid in spPos[l.species]: continue
            chrom,stt,end=spPos[l.species][l.gid]
            if clg_assign[mclg][l.species][chrom]=='': continue
            tet=clg_assign[mclg][l.species][chrom]
            #print l.species,l.gid,spPos[l.species][l.gid],clg,tet,spName[l.species][l.gid]   
            anDup[tet].append(l.name)
    return clgCt,anDup

In [7]:
def checkDupr(t,clg,spPos,clgAsgn):
    # determine most frequent CLG in case several amphioxus genes have distinct clgs
    anDup=defaultdict(list)
    for l in t:
        if l.species in set(['Galgal','Xentro','Chipun','Lepocu']):
            if not l.gid in spPos[l.species]: continue
            chrom,stt,end=spPos[l.species][l.gid]
            #if chrom not in clgAsgn[clg][l.species][chrom]: continue
            tet=clgAsgn[clg][l.species][chrom]
            #print(l.species,l.gid,spPos[l.species][l.gid],clg,tet,spName[l.species][l.gid])
            if not tet==set():
                anDup[tet].append(l.name)
    return anDup

In [8]:
def oriDup(t,anDup,sptr):
    stDup={}
    for nd in anDup:
        if len(anDup[nd])>1:
            stDup[nd]=getAnc(t.get_common_ancestor(anDup[nd]),sptr)
    rDup=[nd[-1] for nd in anDup]
    if all(item in rDup for item in ['1','2']):
        otn=list(chain(*[anDup[nd] for nd in anDup]))
        stDup['12']=getAnc(t.get_common_ancestor(otn),sptr)
    if 'alpha1' in anDup and 'beta1' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha1','beta1']]))
        stDup['ab1']=getAnc(t.get_common_ancestor(otn),sptr)
    if 'alpha2' in anDup and 'beta2' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha2','beta2']]))
        stDup['ab2']=getAnc(t.get_common_ancestor(otn),sptr)
    return stDup

In [9]:
def oriDup12(t,anDup,sptr):
    stDup={}
    #for nd in anDup:
        #if len(anDup[nd])>1:
            #stDup[nd]=getAnc(t.get_common_ancestor(anDup[nd]),sptr)
    rDup=[nd[-1] for nd in anDup]
    if 'alpha1' in anDup or 'beta1' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha1','beta1']]))
        stDup['1']=t.get_common_ancestor(otn)
    if 'alpha2' in anDup or 'beta2' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha2','beta2']]))
        stDup['2']=t.get_common_ancestor(otn)
    return stDup

In [10]:
def oriDupNd(t,anDup,sptr):
    stDup={}
    for nd in anDup:
        if len(anDup[nd])>1:
            stDup[nd]=t.get_common_ancestor(anDup[nd])
    rDup=[nd[-1] for nd in anDup]
    if all(item in rDup for item in ['1','2']):
        otn=list(chain(*[anDup[nd] for nd in anDup]))
        stDup['12']=t.get_common_ancestor(otn)
    if 'alpha1' in anDup and 'beta1' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha1','beta1']]))
        stDup['ab1']=t.get_common_ancestor(otn)
    if 'alpha2' in anDup and 'beta2' in anDup:
        otn=list(chain(*[anDup[nd] for nd in ['alpha2','beta2']]))
        stDup['ab2']=t.get_common_ancestor(otn)
    return stDup

In [11]:
def age(node,sptr):
    tsp=list(set([lf.species for lf in node]))
    if len(tsp)==1:
        return tsp[0]
    else:
        tanc=sptr.get_common_ancestor(tsp).name
        return tanc

In [12]:
def checkCyclo(t,spT):
    spCycl=[l.name for l in spT.search_nodes(name="Cyclostomata")[0].get_leaves()]
    for l in t:
        cla='Cyclostomata' if l.species in spCycl else 'Other'
        l.add_features(clade=cla)
    dec=[]
    acd=[]
    for st in t.get_monophyletic(values=['Cyclostomata'], target_attr="clade"):
        #dupnds=[age(sst) for sst in st.search_nodes(D='Y') if not len(set([l.species for l in sst.get_leaves()]))==1]
        dupnds=[sst for sst in st.search_nodes(D='Y') if not len(set([l.species for l in sst.get_leaves()]))==1]
        #print(dupnds)
        #print(st.get_ascii(attributes=["gid","species", "clade","D"], show_internal=True))
        #ntrees, ndups, sptrees =  st.get_speciation_trees()
        dec.append(len(dupnds))
        for nd in dupnds:
            acd.append([l.name for l in nd.get_leaves()])
        #print(ntrees, ndups)
        #for std in sptrees:
        #    print(std.get_ascii(attributes=["species", "clade"], show_internal=False))
    return dec,acd

#### Load CLG assignment for the 3 species that are in the master table

In [13]:
Leri2Cpun={rc[1]:rc[0] for rc in csv.reader(open('geneInfo/cpun-leri.txt'),delimiter='\t')}

In [14]:
clg_assign=defaultdict( lambda: defaultdict(lambda: defaultdict( set )))

for rc in csv.reader(open('geneInfo/clg_assign_wsk12r.txt'),delimiter='\t'):
    if rc[0]=='CLG': continue
    #print(rc)
    clg,dup,gga,loc,xtr,ler=rc
    clg_assign[clg]['Galgal'][gga.replace('GGA','')].add(dup)
    for lc in loc.split(','):
        clg_assign[clg]['Lepocu'][lc.replace('LOC','LG')].add(dup)
    for lc in ler.split(','):
        if lc in Leri2Cpun:
            cp=Leri2Cpun[lc]
            clg_assign[clg]['Chipun'][cp].add(dup)    
    clg_assign[clg]['Xentro'][xtr.replace('XTR','XTR')].add(dup)

In [15]:
clg_assign2=defaultdict( lambda: defaultdict(lambda: defaultdict( set )))
for clg in clg_assign:
    for sp in clg_assign[clg]:
        for chm in clg_assign[clg][sp]:
            evts='/'.join(list(clg_assign[clg][sp][chm]))
            #if '-' in evts or '-' in chm: continue
            if '/' in evts or '-' in evts or '-' in chm: continue
            #print(clg,sp,chm,evts)
            clg_assign2[clg][sp][chm]=evts

In [16]:
clg_assign2['CLGB']['Xentro']#['XTR6']

defaultdict(set,
            {'XTR6': 'alpha1',
             'XTR10': 'beta1',
             'XTR9': 'alpha2',
             'XTR2': 'beta2'})

In [ ]:
clg_assign2

In [18]:
ckG=set()
with open('geneInfo/Strpur_info.txt','w') as out:
    for r in csv.reader(open("GCF_0000022355_Spur_5_genes.wn.bed"),delimiter='\t'):
        #print(r[13],r[0],r[1],r[2])
        if not r[13] in ckG:
            out.write("{}\t{}\t{}\t{}\n".format(r[13],r[0],r[1],r[2]))
            ckG.add(r[13])
    

#### Load gene positions for the species 

In [19]:
Loc,LocN=genePos('geneInfo/Lepocu_info.txt')
Gga,GgaN=genePos('geneInfo/Galga_info.txt')
Bfl,BflN=genePos('geneInfo/Braflo_info_cor.txt')
Xtr,XtrN=genePos('geneInfo/Xentro_info_v9.txt')
Pma,PmaN=genePos('geneInfo/Petmar2_info.txt')
Pat,PatN=genePos('geneInfo/Pata_info.txt')
Spu,SpuN=genePos('geneInfo/Strpur_info.txt')
Cpu,CpuN=genePos('geneInfo/Chipun_info.txt')


In [20]:
Hsa,HsaN=genePos('geneInfo/Homsap_info.txt')

In [21]:
spPos={'Lepocu':Loc,'Galgal':Gga,'Xentro':Xtr,'Braflo':Bfl,'Petmar':Pma,'Parata':Pat,'Strpur':Spu,'Chipun':Cpu}
spName={'Lepocu':LocN,'Galgal':GgaN,'Braflo':BflN,'Xentro':XtrN}

In [22]:
spT=Tree(open('deut2.tre').read(), format=1)

In [23]:
spPos['Petmar']['POLE']

('chrm10', 16236992, 16320832)

In [24]:
print(list(HsaN.values())[0:10])

['ARHGAP11A', 'AC011295.1', 'NLRP7', 'AP006285.3', 'SURF2', 'GSTT2B', 'CRHR1', 'TSEN34', 'GPX6', 'GP6']


In [25]:
print(len(spPos['Strpur']))

27057


# Check generax trees

read trees from generax and reroot them with Ambulacraria

In [26]:
rTrees={}
nclgbfl=[]
strees={}
nexcl=0
for l in open('brofams_0221_rec.trees'):
    o,ts=l.strip().split(':',1)
    og=o.split('/')[-1].rsplit('_',1)[0]
    #print(og)
    strees[og]=ts
    t=Tree(ts, format=1)
    try:
        t=decoTree(t)
        spl=list(set([l.species for l in t]))
    #print(spl)
        if len(spl)<3: #or 'Braflo' not in spl:
            nexcl+=1
            continue
        rTrees[og]=t
    except:
        print("troubles with {}".format(og))

In [27]:
rTrees['OG_2222']

Tree node 'n449' (0x1da354e8)

In [28]:
decoTree(Tree(strees['OG_2222'],format=1))

Tree node 'n449' (0x1bf84819)

Split gene families that show deep duplications 

In [29]:
verts=set([l.name for l in spT.search_nodes(name='Vertebrata')[0]])
dupDeut=defaultdict(int)
rrTrees={}
j=0
for og,t in rTrees.items():
    j+=1
    vch=[]
    #print(og)
    #print(t.get_ascii(attributes=["species", "name",'D'], show_internal=True))

    for n in t.search_nodes(D='Y'):
        if age(n,spT)=='Deuterostomia':
            #print(n.name)
            #print(n.get_ascii(attributes=["species", "name",'D'], show_internal=True))
            vert_ct,dup_nd=[],[]
            for sn in n.children:
                spl=set([l.species for l in sn])
                nvert=sum([1 if s in verts else 0 for s in spl])
                vert_ct.append(nvert)
                dup_nd.append(sn)
            if vert_ct[0]>0 and vert_ct[1]>0: vch.extend(dup_nd)
    #print(vch)
    if len(vch)>1:
        for i,sn in enumerate(vch,start=1):
            #print(sn.name,sn.get_ascii(attributes=["species", "name",'D'], show_internal=True))
            rrTrees['{}.{}'.format(og,i)]=sn
            
    elif len(vch)==1:
        rrTrees[og]=vch[0]
    else:
        rrTrees[og]=t
    #if j==2: break

In [30]:
print(len(rrTrees))

13612


In [31]:
from copy import deepcopy

In [32]:
clg_assign['CLGK'].items()

dict_items([('Galgal', defaultdict(<class 'set'>, {'1': {'alpha1'}, '4': {'beta1'}, '3': {'alpha2'}, '23': {'beta2'}})), ('Lepocu', defaultdict(<class 'set'>, {'LG3': {'alpha1'}, 'LG7': {'beta1', 'alpha1'}, 'LG1': {'alpha2'}, 'LG6': {'beta2'}})), ('Chipun', defaultdict(<class 'set'>, {'chr6': {'alpha1'}, 'chr12': {'alpha1'}, 'chr15': {'beta1'}, 'chr27': {'beta2'}})), ('Xentro', defaultdict(<class 'set'>, {'XTR2': {'beta2', 'alpha1'}, 'XTR8': {'beta1'}, 'XTR5': {'alpha2'}}))])

In [33]:
list(rrTrees.items())[0:5]


[('OG_10006', Tree node 'n41' (0x1073977c)),
 ('OG_1000', Tree node 'n241' (0x1bf85225)),
 ('OG_1001', Tree node 'n201' (0x1bf82079)),
 ('OG_1002', Tree node 'n145' (0x1bf85240)),
 ('OG_10030', Tree node 'n41' (0x1bf8520a))]

### Make a table for Dan

In [186]:
chrEvtPmar=defaultdict( lambda: defaultdict())

In [187]:
chrEvtPmar

defaultdict(<function __main__.<lambda>()>, {})

In [37]:
chrEvtPmar=defaultdict( lambda: defaultdict())
for r in csv.reader(open('geneInfo/Petmar_ttbl.txt'),delimiter='\t'):
    #print(r)
    clg,ev1,ev2=r
    if not ev2=='-':
        for chrm in ev1.split(','):
            chrEvtPmar[clg][chrm]='1'
        for chrm in ev2.split(','):
            chrEvtPmar[clg][chrm]='2'
    elif ev2=='-':
        for chrm in ev1.split(','):
            chrEvtPmar[clg][chrm]='nd'


In [38]:
chrEvtPata=defaultdict( lambda: defaultdict())
for r in csv.reader(open('geneInfo/Parata_ttbl.txt'),delimiter='\t'):
    #print(r)
    clg,ev1,ev2=r
    if not ev2=='-':
        for chrm in ev1.split(','):
            chrEvtPata[clg][chrm]='1'
        for chrm in ev2.split(','):
            chrEvtPata[clg][chrm]='2'
    elif ev2=='-':
        for chrm in ev1.split(','):
            chrEvtPata[clg][chrm]='nd'


In [190]:
chrEvtPmar['CLGB']['chrm34']

'2'

In [42]:
def findVrtCLG(posV,clgVA=clgVA):
    clgSc=defaultdict(int)
    for i,cs in enumerate(posV):
        for clg in clgVA:
            ic=set(cs).intersection(clgVA[clg][i])
            clgSc[clg]+=len(ic)

    topSc=max(clgSc.values())
    rCLG=[clg for clg in clgSc if clgSc[clg]==topSc]
    if len(rCLG)>2:
        rCLG=[]
    return rCLG

In [36]:
rrTrees['OG_2222']

Tree node 'n449' (0x1e3948eb)

In [ ]:
clg_assign2

In [ ]:
verts=set([l.name for l in spT.search_nodes(name='Vertebrata')[0]])
i=0
out=[]
Pgon={}
oL=[]
for f,t in rrTrees.items():
    at=age(t,spT)
    #spset=set([l.species for l in t])
    spset=set([l.species for l in t])
    BflL=t.search_nodes(species='Braflo')
    #SpuL=t.search_nodes(species='Strpur')
    #spCLG=sum([clgSpur.get(Spu[n.gid][0],[]) for n in SpuL if n.gid in Spu],[])
    bfCLGs=list(set([BflN[l.gid] for l in BflL if not BflN[l.gid]=='NA']))
    SpuL=t.search_nodes(species='Strpur')
    spCLGs=list(set(sum([clgSpur.get(Spu[n.gid][0],[]) for n in SpuL if n.gid in Spu],[])))
    #print(bfCLGs,spCLGs)
    Hsap=[l.gid for l in t.search_nodes(species='Homsap')]
    #HsapN=[HsaN[g] for g in Hsap if g.startswith('ENS')]
    if len(spset.intersection(verts))==0: continue
    #for sp in ['Homsap','Lepocu','Galgal','Petmar','Parata']:
    posV=[]
    for sp in ['Chipun','Lepocu','Xentro','Galgal']:
        #print(sp,[spPos[sp][l.gid][0] for l in t.search_nodes(species=sp)])
        posV.append([spPos[sp][l.gid][0] for l in t.search_nodes(species=sp) if l.gid in spPos[sp]])
    rvCLGs=findVrtCLG(posV)
    #print(posV)
    #print(f,bfCLGs,spCLGs,rvCLGs)
    #print(set(bfCLGs) & set(spCLGs) & set(rvCLGs))
    #print(set(bfCLGs) & set(rvCLGs))
    #print(Counter(bfCLGs+spCLGs))
    #clg=Counter(bfCLGs+spCLGs+rvCLGs).most_common()[0][0] if len(Counter(bfCLGs+spCLGs+rvCLGs))>0 else clg=='nd'
    clg='nd'
    if len(Counter(bfCLGs+spCLGs))>0:
        clg=Counter(bfCLGs+spCLGs).most_common()[0][0] 
    elif len(Counter(rvCLGs))>0:
        clg=Counter(rvCLGs).most_common()[0][0]
        print(f,bfCLGs,spCLGs,rvCLGs,clg,Counter(rvCLGs).most_common())

        
    #clgSp=Counter(spCLGs).most_common()[0][0]
    if clg=='CLGA':
        if 'CLGA1' in spCLGs or 'CLGA1' in rvCLGs:
            clg='CLGA1'
        elif 'CLGA2' in spCLGs or 'CLGA2' in rvCLGs:
            clg='CLGA2'
        else:
            clg='nd'
    #print(clg,bfCLGs,spCLGs,rvCLGs)
    #print(Counter(bfCLGs+spCLGs+rvCLGs).most_common()[0])
    #print(clg)
        
    #Locu=[l.gid for l in t.search_nodes(species='Lepocu')]
    #Ggal=[l.gid for l in t.search_nodes(species='Galgal')]
    Pmar=[l.gid for l in t.search_nodes(species='Petmar')]
    Pata=[l.gid for l in t.search_nodes(species='Parata')]

    #print(Pmar)
    gnthGnEv={}
    if not clg=='nd':
        anDup=checkDupr(t,clg,spPos,clg_assign2)
        for e,l in anDup.items():
            for g in l:
                gnthGnEv[g]=e
                #sp,gid=g.split('_',1)
                #print(f,clg,','.join(rvCLGs),e,sp,gid,spName[sp].get(gid,'nf'),pos[0],pos[1])
    #print(anDup)
    #print(str(clg))
    rvDup={}   
    for g in Pmar:
        pos=spPos['Petmar'].get(g,('',0,0))
        pevt=chrEvtPmar[clg].get(pos[0],'nd')#.get(pos[0],['nd','nd'])
        sol=[f,clg,'-',pevt,'Petmar',g,'NA',pos[0],pos[1]]
        #print(sol)
        oL.append(sol)

    for g in Pata:
        pos=spPos['Parata'].get(g,('',0,0))
        pevt=chrEvtPata[clg].get(pos[0],'nd')
        sol=[f,clg,'-',pevt,'Parata',g,'NA',pos[0],pos[1]]
        #print(sol)
        oL.append(sol)
    #print(oL)
    for l in t:
        sp,gid=l.species,l.gid
        #sp,gid=g.split('_',1)
        if sp in ['Xentro','Lepocu','Galgal','Chipun']:
            pos=spPos[sp].get(gid,('',0,0))
            spn=spName[sp].get(gid,'nf') if sp in ['Xentro','Lepocu','Galgal'] else 'nd'
            evt=gnthGnEv.get(l.name,'nd')
            if clg=="nd":
                #print(anDup)
                print(f,clg,bfCLGs,spCLGs,rvCLGs,evt,sp,gid,spn,pos[0],pos[1],gnthGnEv)

            oL.append([f,clg,','.join(rvCLGs),evt,sp,gid,spn,pos[0],pos[1]])
            #rvDup[]
    #print(f,clg,at,','.join(list(HsapN)),anDup,HsaP,Hsap,Locu,Ggal)
    #print(anDup)
   # Pgon[f]=frozenset(anDup.keys())
    #pdp=set([p[-1] for p in anDup.keys()])
    #if len(anDup)<3 or len(pdp)==1: continue
    i+=1
    #if i==50: break
    

In [ ]:
oL

In [63]:
clg_assign2['CLGB']['Xentro']

defaultdict(set,
            {'XTR6': 'alpha1',
             'XTR10': 'beta1',
             'XTR9': 'alpha2',
             'XTR2': 'beta2',
             'XTR1': set()})

In [46]:
print(list(spPos['Parata'])[0:5])

['EA00001', 'EA00003', 'EA00006', 'EA00007', 'EA00009']


In [92]:
testv=[['chr25'], ['LG20'], ['XTR1'], ['15', '15']]

In [97]:
findVrtCLG(testv,clgVA=clgVA)

[]

In [95]:
clgSc=defaultdict(int)
for i,cs in enumerate(testv):
    for clg in clgVA:
        ic=set(cs).intersection(clgVA[clg][i])
        clgSc[clg]+=len(ic)
topSc=max(clgSc.values())
rCLG=[clg for clg in clgSc if clgSc[clg]==topSc]
print(clgSc,rCLG)

defaultdict(<class 'int'>, {'CLGC': 1, 'CLGL': 1, 'CLGM': 0, 'CLGE': 0, 'CLGO': 0, 'CLGI': 1, 'CLGQ': 1, 'CLGF': 1, 'CLGN': 0, 'CLGP': 0, 'CLGA1': 0, 'CLGA2': 0, 'CLGK': 0, 'CLGJ': 0, 'CLGD': 0, 'CLGB': 4, 'CLGG': 4, 'CLGH': 0, 'CLG': 0}) ['CLGB', 'CLGG']


In [207]:
print(len(set([l[0] for l in oL])))

11254


In [ ]:
clg_assign

In [74]:
oLs=sorted(oL,key=lambda x:float(x[0].split('_')[1]))

In [75]:
hL=['FID','CLG','CLGv','1R','Sp','GID','Gname','chrm','Pos']

In [76]:
df = pd.DataFrame(oLs, columns = hL)

In [77]:
df#[df.Sp=='Chipun']

,FID,CLG,CLGv,1R,Sp,GID,Gname,chrm,Pos
0,OG_1,CLGP,-,nd,Petmar,POLR2H,NA,chrm5,5962455
1,OG_1,CLGP,-,nd,Parata,EA33602,NA,chr7,4558168
2,OG_1,CLGP,"CLGN,CLGP",alpha2,Galgal,ENSGALG00000008534,POLR2H,9,15996145
3,OG_1,CLGP,"CLGN,CLGP",alpha2,Xentro,polr2h,Xetrov90014686m,XTR5,111547748
4,OG_1,CLGP,"CLGN,CLGP",alpha2,Lepocu,ENSLOCG00000008946,polr2h,LG14,18263274
...,...,...,...,...,...,...,...,...,...
105846,OG_22217,nd,,nd,Lepocu,ENSLOCG00000014028,zgc:172079,LG2,61165908
105847,OG_22217,nd,,nd,Xentro,LOC100493299,Xetrov90029005m,scaffold_48,508560
105848,OG_22217,nd,,nd,Xentro,LOC100493138,Xetrov90029004m,scaffold_48,483093
105849,OG_22278,CLGF,,alpha1,Lepocu,ENSLOCG00000008308,hdac12,LG3,13665151


In [78]:
df.to_csv('Vert_Evt_OGrrA.txt',index=False,sep='\t')

In [83]:
df

,FID,CLG,CLGv,1R,Sp,GID,Gname,chrm,Pos
0,OG_1,CLGP,CLGG,1,Petmar,POLR2H,NA,chrm5,5962455
1,OG_1,CLGP,CLGO,2,Parata,EA33602,NA,chr7,4558168
2,OG_2,CLGO,CLGJ,2,Petmar,COPB1,NA,chrm3,17170487
3,OG_2,CLGO,CLGM,nd,Parata,EA45752,NA,chr2,180728977
4,OG_3,CLGA1,CLGA1,2,Petmar,YIPF4,NA,chrm6,3963531
...,...,...,...,...,...,...,...,...,...
34147,OG_21678,CLGF,"CLGF,CLGN",alpha1,Galgal,ENSGALG00000042735,,1,92419270
34148,OG_21678,CLGF,"CLGF,CLGN",alpha1,Galgal,ENSGALG00000039452,GJA5,1,92426541
34149,OG_21777,CLGF,CLGF,1,Petmar,LOC116955772,NA,chrm61,8185934
34150,OG_21777,CLGF,CLGF,alpha1,Galgal,ENSGALG00000019145,,1,130465409


In [60]:
with open('Vert_Evt_OGr2.txt','w') as out: 
    out.write('\t'.join(hL)+'\n')
    for r in oLs:
        out.write('\t'.join(map(str,r))+'\n')

# New approach to molecular dating 

In [24]:
def checkDup(t,spPos):
    # determine most frequent CLG in case several amphioxus genes have distinct clgs
    BflTax=[l for l in t if l.species=='Braflo']        
    #clgs=[BflN[l.gid] for l in BflTax]
    clgs=sum([SpuN[l.gid] for l in SpuL],[])
    clgCt=Counter(clgs)
    #print(clgCt)
    mclg=clgCt.most_common(1)[0][0]
    anDup=defaultdict(list)
    for l in t:
        if l.species in spPos:
            if not l.gid in spPos[l.species]: continue
            chrom,stt,end=spPos[l.species][l.gid]
            if clg_assign[mclg][l.species][chrom]=='': continue
            tet=clg_assign[mclg][l.species][chrom]
            #print l.species,l.gid,spPos[l.species][l.gid],clg,tet,spName[l.species][l.gid]   
            anDup[tet].append(l.name)
    return mclg,anDup

In [25]:
clgPmar,clgPata=defaultdict(set),defaultdict(set)
for r in csv.reader(open("Pata_Pmar_clg.txt"),delimiter='\t'):
    if r[0]=='s2chr': continue
    #clg='A' if r[2]=='A1' else r[2]
    clg=r[2]
    clgPmar['CLG'+clg].add(r[1])
    clgPata['CLG'+clg].add(r[0])
   
    #print(r)

In [ ]:
clg_assign

In [40]:
clgSpur=defaultdict(list)
for r in csv.reader(open("Strpur_clgs.txt"),delimiter='\t'):
    clgs=['CLG'+c if len(c)==1 or c in ['A1','A2'] else 'CLG'+c[0] for c in r[1].split(',') ]
    print(r[0],clgs)
    clgSpur[r[0]].extend(clgs)

scaffold_1 ['CLGA2']
scaffold_2 ['CLGG']
scaffold_3 ['CLGL']
scaffold_4 ['CLGM']
scaffold_5 ['CLGA1']
scaffold_6 ['CLGC', 'CLGB', 'CLGE']
scaffold_7 ['CLGJ', 'CLGB']
scaffold_8 ['CLGD']
scaffold_9 ['CLGC']
scaffold_10 ['CLGQ']
scaffold_11 ['CLGF']
scaffold_12 ['CLGN']
scaffold_13 ['CLGJ']
scaffold_14 ['CLGH']
scaffold_15 ['CLGI']
scaffold_16 ['CLGP']
scaffold_17 ['CLGO']
scaffold_18 ['CLGO']
scaffold_19 ['CLGB']
scaffold_20 ['CLGR']
scaffold_21 ['CLGK']


In [ ]:
SpuN=defaultdict(list)
for g,l in Spu.items():
    print(g,l[0],clgSpur[l[0]])
    SpuN[g]=clgSpur.get(l[0],'')
    #break

In [ ]:
checkDup(rrTrees['OG_11749'],spPos)

In [ ]:
print(rrTrees['OG_1002'])

In [102]:
spPos.keys()

dict_keys(['Lepocu', 'Galgal', 'Xentro', 'Braflo', 'Petmar', 'Parata', 'Strpur', 'Chipun'])

In [52]:
def checkDupr(t,clg,spPos,clgAsgn):
    anDup=defaultdict(list)
    for l in t:
        if l.species in {'Lepocu', 'Galgal', 'Xentro','Chipun'}:
            if not l.gid in spPos[l.species]: continue
            chrom,stt,end=spPos[l.species][l.gid]
            if not chrom in clgAsgn[clg][l.species]: continue
            tet=clgAsgn[clg][l.species][chrom]
            #print(l.species,l.gid,spPos[l.species][l.gid],clg,tet)
            anDup[tet].append(l.name)
    return anDup

In [147]:
checkDupr(rrTrees['OG_1677'],'CLGQ',spPos,clg_assign2)

defaultdict(list, {})

In [96]:
clg_assign2['CLGK']['Xentro']

'alpha2'

In [201]:
bfCLG=[BflN[l.gid] for l in rrTrees['OG_20160'].search_nodes(species='Braflo')]

In [203]:
[clgSpur.get(Spu[n.gid][0],[]) for n in rrTrees['OG_20160'].search_nodes(species='Strpur')]

[['CLGA1']]

In [202]:
print(bfCLG)

['CLGA']


In [199]:
pos=spPos['Petmar'].get('LHX1',('',0,0))
chrEvtPmar['CLGB'].get('chrm38','nd')


'nd'

In [92]:
chrEvtPmar['CLGL'].get('chrm44','nd')

'nd'

In [94]:
chrEvtPata['chr7'].get('chr10','nd')

'nd'

In [85]:
checkDupr(rrTrees['OG_1677'],'CLGM',spPos,clg_assign2)

defaultdict(list,
            {'alpha1': ['Lepocu_ENSLOCG00000004020',
              'Xentro_dmbx1',
              'Galgal_ENSGALG00000010436',
              'Chipun_Chipu0005518']})

In [205]:
clg='CLGA'
for l in rrTrees['OG_4927']:
    if l.species in set(['Galgal','Xentro','Chipun','Lepocu']):
        if not l.gid in spPos[l.species]: continue
        chrom,stt,end=spPos[l.species][l.gid]
        print(l.species,l.gid,spPos[l.species][l.gid],clg_assign2[clg][l.species][chrom])
        #if not chrom in clg_assign2[clg][l.species][chrom]: continue
        tet=clg_assign2[clg][l.species][chrom]
        print(l.species,l.name,l.gid,spPos[l.species][l.gid],clg,tet,spName.get('l.species','nd'))


Chipun Chipu0010645 ('chr25', 31254172, 31261253) set()
Chipun Chipun_Chipu0010645 Chipu0010645 ('chr25', 31254172, 31261253) CLGA set() nd
Lepocu ENSLOCG00000008328 ('LG20', 14084503, 14115002) set()
Lepocu Lepocu_ENSLOCG00000008328 ENSLOCG00000008328 ('LG20', 14084503, 14115002) CLGA set() nd
Xentro lhx5 ('XTR1', 133703287, 133726520) set()
Xentro Xentro_lhx5 lhx5 ('XTR1', 133703287, 133726520) CLGA set() nd
Galgal ENSGALG00000008303 ('15', 12639428, 12644631) set()
Galgal Galgal_ENSGALG00000008303 ENSGALG00000008303 ('15', 12639428, 12644631) CLGA set() nd
Lepocu ENSLOCG00000000357 ('AHAT01043974.1', 1099, 4145) set()
Lepocu Lepocu_ENSLOCG00000000357 ENSLOCG00000000357 ('AHAT01043974.1', 1099, 4145) CLGA set() nd
Lepocu ENSLOCG00000004960 ('LG22', 7558717, 7567347) set()
Lepocu Lepocu_ENSLOCG00000004960 ENSLOCG00000004960 ('LG22', 7558717, 7567347) CLGA set() nd
Xentro lhx1 ('XTR2', 49862098, 49873565) set()
Xentro Xentro_lhx1 lhx1 ('XTR2', 49862098, 49873565) CLGA set() nd
Galgal E

In [ ]:
clg_assign2[clg]['Chipun']

In [263]:
print(rrTrees['OG_2892'].write(format=1))

((((Ptyfla_pfl_40v0_9_20150316_1g31822:0.106164,Ptyfla_pfl_40v0_9_20150316_1g25097:1e-06)100:0.329348,Sackow_Sakowv30012058m:0.390453)100:0.66229,(((Strpur_LOC579970:0.019736,Strpur_LOC762569:0.084777)100:0.059386,(Strpur_LOC115923164:0.008152,Strpur_LOC100893855:0.004053)100:0.012219)100:1.2917,Acapla_LOC110982062:1.02927)100:0.279163)100:0.512013,(((((((Eptbur_Eptbu0034081:0.093773,Parata_EA36646:0.008609)100:0.571272,(Eptbur_Eptbu0034084:0.206781,Parata_EA37041:1e-06)100:1.83406)100:0.637158,(Petmar_LOC116947536:0.011558,((Letjap_g6222:0.080164,Letrei_Hic_chr_38_136:0.107337)98:0.060352,(Letjap_g6223:0.779313,Letrei_Hic_chr_38_138:0.081709)100:0.465668)100:1e-06)100:0.960052)100:0.541538,((((Eptbur_Eptbu0020379:0.066244,Parata_EA79398:1e-06)100:0.381094,(Eptbur_Eptbu0026466:0.004714,Parata_EA73963:0.477391)95:1e-06)100:0.765906,(Petmar_LOC116952791:1e-06,(((Letrei_Hic_chr_55_21:0.063678,Letrei_Hic_chr_55_213:0.158428)91:0.01043,Letjap_g15697:0.139584)92:0.15712,(Letjap_g15696:0.2153

In [262]:
print(rrTrees['OG_2892'])


            /-Ptyfla_pfl_40v0_9_20150316_1g31822
         /-|
      /-|   \-Ptyfla_pfl_40v0_9_20150316_1g25097
     |  |
     |   \-Sackow_Sakowv30012058m
     |
   /-|         /-Strpur_LOC579970
  |  |      /-|
  |  |     |   \-Strpur_LOC762569
  |  |   /-|
  |  |  |  |   /-Strpur_LOC115923164
  |   \-|   \-|
  |     |      \-Strpur_LOC100893855
  |     |
  |      \-Acapla_LOC110982062
  |
  |                     /-Eptbur_Eptbu0034081
  |                  /-|
  |                 |   \-Parata_EA36646
  |               /-|
  |              |  |   /-Eptbur_Eptbu0034084
  |              |   \-|
  |            /-|      \-Parata_EA37041
  |           |  |
  |           |  |   /-Petmar_LOC116947536
  |           |  |  |
  |           |   \-|      /-Letjap_g6222
  |           |     |   /-|
  |           |     |  |   \-Letrei_Hic_chr_38_136
  |           |      \-|
  |           |        |   /-Letjap_g6223
  |           |         \-|
  |           |            \-Letrei_Hic_chr_38_138
  |     

In [116]:
i=0
out=[]
Pgon={}
for f,t in rrTrees.items():
    i+=1
    #if i==10: break
    at=age(t,spT)
    #spset=set([l.species for l in t])
    spset=set([l.species for l in t])
    if not 'Braflo' in spset or not len(spset.intersection(verts))>0: continue   
    BflL=t.search_nodes(species='Braflo')
    SpuL=t.search_nodes(species='Strpur')
    spCLG=sum([clgSpur.get(Spu[n.gid][0],[]) for n in SpuL if n.gid in Spu],[])
    bfCLG=[BflN[l.gid] for l in BflL]
    #clgs=sum([SpuN[l.gid] for l in SpuL],[])
    #print(clgs)
    #print(set(clgs).intersection(set(['A1','A2'])))
    clg=Counter(spCLG+bfCLG).most_common()[0][0]
    #if not clg=='CLGB': continue
    #if len(set(clgs).intersection(set(['CLGG'])))==0:
    #    continue
    #print(clgs)
    SkoL=[l.gid for l in t.search_nodes(species='Sackow')]+['']
    PflL=[l.gid for l in t.search_nodes(species='Ptyfla')]+['']
    AplL=[l.gid for l in t.search_nodes(species='Acapla')]+['']
    BlaL=[l.gid for l in t.search_nodes(species='Bralan')]+['']
    
    anDup=checkDupr(t,clg,spPos,clg_assign2)
    Pgon[f]=frozenset(anDup.keys())
    pdp=set([p[-1] for p in anDup.keys()])
    #print(i,f,clg,pdp,anDup.keys(),anDup)
    if len(anDup)<2 or len(pdp)==1: continue # more relaxed param for A1 (and A2)
    #if len(anDup)<3 or len(pdp)==1: continue
    rSpu=[l.gid for l in SpuL if clg in SpuN[l.gid]]
    rSpu.append('')
#   print(clg,rSpu,)
    rBfl=[l.gid for l in BflL if BflN[l.gid]==clg]
    rBfl.append('')
    pggn={}
    for pgn in ['alpha1','beta1','alpha2','beta2']:
        pggn[pgn]={}
        for g in anDup.get(pgn,[]):
            sp,gn=g.rsplit('_',1)
            pggn[pgn][sp]=gn
    #print(pggn)
    #print([f,clg,rBfl,rSpu,BlaL,SkoL,PflL,AplL])
    oL=[f,clg,rBfl[0],rSpu[0],BlaL[0],SkoL[0],PflL[0],AplL[0]]
    #hd=['fid','clg','Bflo','Lepocu','Xentro','Galgal','Lepocu','Chipun','Parata']
    spG=defaultdict(list)
    for sp in ['Chipun','Lepocu','Xentro','Galgal']:
        for pgn in ['alpha1','beta1','alpha2','beta2']:
            if sp in pggn[pgn]:
                spG[sp].append("{}_{}|{}".format(sp,pggn[pgn][sp],pgn))
                #print("{}_{}|{}".format(sp,pggn[pgn][sp],pgn))
            #print(pgn,sp,pggn[pgn].get(sp,''))
    #hd.append("{}_{}".format(pgn,sp))
    #oL.append(pggn[pgn].get(sp,''))
    #print(spG)
    for l in t.search_nodes(species='Petmar'):
        pos=spPos['Petmar'][l.gid][0]
        if pos in clgPmar[clg]:
            spG['Petmar'].append("{}_{}|{}".format('Petmar',l.gid,pos))
    for l in t.search_nodes(species='Parata'):
        pos=spPos['Parata'].get(l.gid,[0])[0]
        if pos in clgPata[clg]:
            spG['Parata'].append("{}_{}|{}".format('Parata',l.gid,pos))
            #print("{}_{}|{}".format('Parata',l.gid,pos))
    #print(PmaL)
    #PatL=t.search_nodes(species='Parata')
    for sp in ['Chipun','Lepocu','Xentro','Galgal','Lepocu','Petmar','Parata']:
        oL.append(';'.join(spG[sp]))
    #print(hd)
    out.append(oL)
    #i+=1
    

In [ ]:
print(out)

In [117]:
print(len(out))

1823


In [118]:
hd=['fid','clg','Bflo','Spu','Blan','Skow','Pfla','Apla','Chipun','Lepocu','Xentro','Galgal','Lepocu','Petmar','Parata']

In [119]:
phyloclg=pd.DataFrame(out, columns =hd)

In [121]:
phyloclg.sort_values(by=['clg'], inplace=True)


In [122]:
phyloclg

,fid,clg,Bflo,Spu,Blan,Skow,Pfla,Apla,Chipun,Lepocu,Xentro,Galgal,Lepocu,Petmar,Parata
1395,OG_6628,CLGA1,,LOC578715,BL09586,Sakowv30006424m,pfl_40v0_9_20150316_1g10237,LOC110980554,Chipun_Chipu0002554|alpha1;Chipun_Chipu0012983...,Lepocu_ENSLOCG00000009500|alpha1;Lepocu_ENSLOC...,Xentro_egln2|alpha1;Xentro_egln1|alpha2,Galgal_ENSGALG00000010001|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000009500|alpha1;Lepocu_ENSLOC...,Petmar_LOC116953961|chrm53;Petmar_LOC116947785...,Parata_EA18103|chr13;Parata_EA12406|chr10;Para...
367,OG_1700,CLGA1,,LOC578865,BL02832,Sakowv30002459m,,LOC110976744,Chipun_Chipu0020963|alpha2,Lepocu_ENSLOCG00000016222|alpha2,Xentro_kcnk6|alpha1,Galgal_ENSGALG00000011005|alpha2,Lepocu_ENSLOCG00000016222|alpha2,Petmar_LOC116939679|chrm6;Petmar_LOC116948302|...,Parata_EA47364|chr9;Parata_EA38796|chr1
51,OG_1150,CLGA1,,LOC575813,BL06663,Sakowv30013194m,pfl_40v0_9_20150316_1g20928,LOC110975166,Chipun_Chipu0011142|alpha2,Lepocu_ENSLOCG00000016285|alpha2,Xentro_c6orf211|beta1,Galgal_ENSGALG00000026322|alpha2,Lepocu_ENSLOCG00000016285|alpha2,Petmar_ARMT1|chrm6,Parata_EA47347|chr9
823,OG_2751,CLGA1,,LOC580795,BL21129,Sakowv30031205m,pfl_40v0_9_20150316_1g9151,LOC110979503,Chipun_Chipu0011948|alpha1;Chipun_Chipu0022910...,Lepocu_ENSLOCG00000010222|alpha1;Lepocu_ENSLOC...,Xentro_ppp2r5e|alpha1;Xentro_ppp2r5a|alpha2,Galgal_ENSGALG00000011840|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000010222|alpha1;Lepocu_ENSLOC...,Petmar_LOC116947244|chrm29;Petmar_PPP2R5E|chrm...,Parata_EA53345|chr14;Parata_EA48022|chr9
945,OG_3571,CLGA1,,LOC115929506,BL15153,Sakowv30033198m,pfl_40v0_9_20150316_1g12625,LOC110979502,Chipun_Chipu0012844|alpha2,Lepocu_ENSLOCG00000008909|alpha1;Lepocu_ENSLOC...,Xentro_Xetrov90021168m|alpha1;Xentro_lhcgr|alpha2,Galgal_ENSGALG00000010572|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000008909|alpha1;Lepocu_ENSLOC...,Petmar_LOC103091684|chrm33;Petmar_LOC103091775...,Parata_EA36232|chr1;Parata_EA11448|chr10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,OG_17281,CLGQ,BF15228,LOC580376,BL00006,Sakowv30045756m,pfl_40v0_9_20150316_1g8026,LOC110977987,Chipun_Chipu0001412|beta1;Chipun_Chipu0008223|...,Lepocu_ENSLOCG00000009344|alpha1;Lepocu_ENSLOC...,Xentro_tet2|alpha2,Galgal_ENSGALG00000031604|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000009344|alpha1;Lepocu_ENSLOC...,Petmar_LOC116950889|chrm41,Parata_EA77789|chr3;Parata_EA79111|chr3
1127,OG_496,CLGQ,BF15386,LOC100892398,BL03951,Sakowv30022063m,pfl_40v0_9_20150316_1g6979,LOC110977562,Chipun_Chipu0015445|alpha1;Chipun_Chipu0012572...,Lepocu_ENSLOCG00000012923|alpha1;Lepocu_ENSLOC...,Xentro_tspan14|alpha1;Xentro_tspan5|alpha2,Galgal_ENSGALG00000002414|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000012923|alpha1;Lepocu_ENSLOC...,Petmar_LOC116940233|chrm8,Parata_EA04379|chr6
599,OG_19251,CLGQ,BF15432,LOC590057,BL24628,Sakowv30008992m,pfl_40v0_9_20150316_1g29158,LOC110981231,Chipun_Chipu0001501|beta1,Lepocu_ENSLOCG00000015345|beta1;Lepocu_ENSLOCG...,Xentro_cds1|alpha2,Galgal_ENSGALG00000030406|beta1;Galgal_ENSGALG...,Lepocu_ENSLOCG00000015345|beta1;Lepocu_ENSLOCG...,Petmar_LOC116950985|chrm41;Petmar_CDS2|chrm8,Parata_EA03551|chr6
438,OG_17804,CLGQ,BF26773,LOC582657,BL95131,Sakowv30014963m,pfl_40v0_9_20150316_1g20443,LOC110973998,Chipun_Chipu0015568|alpha1;Chipun_Chipu0002365...,Lepocu_ENSLOCG00000005214|alpha1;Lepocu_ENSLOC...,Xentro_sar1a|alpha1,Galgal_ENSGALG00000004431|alpha1;Galgal_ENSGAL...,Lepocu_ENSLOCG00000005214|alpha1;Lepocu_ENSLOC...,,Parata_EA00504|chr6


In [57]:
print(len(out))

788


In [123]:
phyloclg.to_csv('paralogons_genes_rsGrRx2.txt',sep='\t',index=False)

In [124]:
print(Counter(Pgon.values()))

Counter({frozenset(): 2220, frozenset({'alpha1'}): 1952, frozenset({'alpha2'}): 1829, frozenset({'alpha2', 'alpha1'}): 898, frozenset({'beta1'}): 497, frozenset({'beta2'}): 448, frozenset({'beta1', 'alpha1'}): 291, frozenset({'alpha1', 'alpha2', 'beta1'}): 273, frozenset({'beta2', 'alpha2'}): 198, frozenset({'beta2', 'alpha2', 'alpha1'}): 186, frozenset({'beta2', 'alpha2', 'beta1', 'alpha1'}): 133, frozenset({'alpha2', 'beta1'}): 118, frozenset({'beta2', 'alpha1'}): 87, frozenset({'beta2', 'beta1'}): 46, frozenset({'beta2', 'alpha2', 'beta1'}): 45, frozenset({'alpha1', 'beta2', 'beta1'}): 37})


# Check trees

In [23]:
rrTreesAn=deepcopy(rrTrees)

In [24]:
def extDup(anDup):
    exdup={}
    if 'alpha1'  in anDup and 'beta1' in anDup:
        exdup['ab1']=anDup['alpha1']+anDup['beta1']
    if 'alpha2'  in anDup and 'beta2' in anDup:
        exdup['ab2']=anDup['alpha2']+anDup['beta2']
    if all(item in [nd[-1] for nd in anDup] for item in ['1','2']):
        exdup['12']=list(set(chain(*[anDup[nd] for nd in anDup])))
    return exdup

In [25]:
testt=rrTreesAn['OG_11749']

In [26]:
print([l.name for l in spT.search_nodes(name='Vertebrata')[0]])

['Letjap', 'Letrei', 'Petmar', 'Parata', 'Eptbur', 'Calmil', 'Chipun', 'Lepocu', 'Amical', 'Danrer', 'Orylat', 'Latcha', 'Xentro', 'Galgal', 'Homsap', 'Musmus']


In [41]:
i=0
dupNds=defaultdict(int)
pats=[]
patDs=[]
extpats=[]
verts=set([l.name for l in spT.search_nodes(name='Vertebrata')[0]])
for fid,t in rrTreesAn.items():
    spset=set([l.species for l in t])
    if not 'Braflo' in spset or not len(spset.intersection(verts))>0: continue
    i+=1
    clgct,anDup=checkDup(t,spPos)
    edts=extDup(anDup)
    pats.append(list(edts.keys()))
    eda={de:age(t.get_common_ancestor(det),spT) for de,det in edts.items()}
    edd={de:t.get_common_ancestor(det).D for de,det in edts.items() if t.get_common_ancestor(det).D=='Y'}
    #pats.append(eda.values())
    patDs.append(list(edd.keys()))
    extpats.append(eda)
    #print(fid)
    for dn in t.search_nodes(D='Y'):
        dupNds[age(dn,spT)]+=1
    #if i==10: break 

print(dupNds)
#print(extpats)

defaultdict(<class 'int'>, {'Strpur': 7346, 'Ptyfla': 6506, 'Myxines': 3011, 'Eptbur': 2721, 'Letrei': 1066, 'Orylat': 1353, 'Galgal': 1143, 'Letjap': 4664, 'Deuterostomia': 2707, 'Bralan': 4946, 'Vertebrata': 3423, 'Euteleostei': 2406, 'Gnathostomata': 4441, 'Homsap': 931, 'Sackow': 4168, 'Braflo': 3119, 'Musmus': 1014, 'Chipun': 1832, 'Mammalia': 552, 'Cyclostomata': 1675, 'Danrer': 2023, 'Cephalochordata': 7255, 'Hyperoartia': 1868, 'Hemichordata': 1801, 'Parata': 1482, 'Xentro': 2587, 'Acapla': 2179, 'Brabel': 3276, 'Chordata': 798, 'Ceph2': 1062, 'Latcha': 911, 'Lepocu': 479, 'Echinodermata': 618, 'Actinopterygii': 642, 'Leth': 924, 'Chondrichthyes': 526, 'Osteichthyes': 764, 'Sarcopterygia': 246, 'Tetrapoda': 280, 'Ambulacraria': 581, 'Calmil': 441, 'Petmar': 514, 'Amniotes': 156})


In [37]:
print(dupNds)

defaultdict(<class 'int'>, {'Strpur': 7346, 'Ptyfla': 6506, 'Myxines': 3011, 'Eptbur': 2721, 'Letrei': 1066, 'Orylat': 1353, 'Galgal': 1143, 'Letjap': 4664, 'Deuterostomia': 2707, 'Bralan': 4946, 'Vertebrata': 3423, 'Euteleostei': 2406, 'Gnathostomata': 4441, 'Homsap': 931, 'Sackow': 4168, 'Braflo': 3119, 'Musmus': 1014, 'Chipun': 1832, 'Mammalia': 552, 'Cyclostomata': 1675, 'Danrer': 2023, 'Cephalochordata': 7255, 'Hyperoartia': 1868, 'Hemichordata': 1801, 'Parata': 1482, 'Xentro': 2587, 'Acapla': 2179, 'Brabel': 3276, 'Chordata': 798, '': 1062, 'Latcha': 911, 'Lepocu': 479, 'Echinodermata': 618, 'Actinopterygii': 642, 'Leth': 924, 'Chondrichthyes': 526, 'Osteichthyes': 764, 'Sarcopterygia': 246, 'Tetrapoda': 280, 'Ambulacraria': 581, 'Calmil': 441, 'Petmar': 514, 'Amniotes': 156})


In [45]:
print(Counter(['_'.join(sorted(p)) for p in pats]))

Counter({'': 6817, '12': 1124, 'ab2': 378, '12_ab2': 332, '12_ab1': 249, 'ab1': 207, '12_ab1_ab2': 151})


In [47]:
print(Counter(['_'.join(sorted(p)) for p in patDs]))

Counter({'': 7187, '12': 1076, '12_ab1': 252, '12_ab2': 239, 'ab1': 185, 'ab2': 184, '12_ab1_ab2': 134, 'ab1_ab2': 1})


In [ ]:
print(print(testt.get_ascii(attributes=["D","name","Clade"], show_internal=True)))

# Generate a table

In [32]:
oriRes={}
famEvents={}
hasCyclo={}
clgCts={}
cyclostomes=set(['Parata','Eptbur','Letrei','Petmar','Letjap'])

nd12={}
i=0
for fid,ftr in rrTrees.items():
    #print(fid)
    spset=set([l.species for l in ftr])
    if not 'Braflo' in spset or not len(spset.intersection(verts))>0: continue
    clgct,anDup=checkDup(ftr,spPos)
    #dec,acd=checkCyclo(ftr,spT)
    clgCts[fid]=clgct
    oridup=oriDup(ftr,anDup,spT)
    famEvents[fid]=anDup
    #print(fid,anDup,oridup)
    glf=[l.name for l in ftr if l.species in cyclostomes]
    hasCyclo[fid]=len(glf)
    #print fid,anDup,oridup
    oriRes[fid]=oridup
    ori12=oriDup12(ftr,anDup,spT)
    #print(ori12)
    if len(ori12)>0:
        n12=ftr.get_common_ancestor(list(ori12.values()))
        nd12[fid]=n12.D
    else:
        nd12[fid]='NA'
    #print(n12.name,n12.D)
    i+=1
    #if i==10: break
   

In [146]:
print(anDup)
print(list(famEvents.items())[0:10])
print(list(oriRes.items())[0:10])

defaultdict(<class 'list'>, {'alpha2': ['Lepocu_ENSLOCG00000007566', 'Xentro_sf3b3', 'Galgal_ENSGALG00000002531'], 'beta2': []})
[('OG_1000', defaultdict(<class 'list'>, {})), ('OG_1001', defaultdict(<class 'list'>, {'beta2': ['Xentro_mau2', 'Galgal_ENSGALG00000002969', 'Lepocu_ENSLOCG00000004881'], 'alpha2': []})), ('OG_1002', defaultdict(<class 'list'>, {})), ('OG_1003.1', defaultdict(<class 'list'>, {'alpha2': ['Lepocu_ENSLOCG00000012369', 'Galgal_ENSGALG00000026195'], 'beta2': []})), ('OG_1004', defaultdict(<class 'list'>, {'alpha1': ['Xentro_rpl4', 'Galgal_ENSGALG00000007711', 'Lepocu_ENSLOCG00000013867'], 'beta1': []})), ('OG_1006', defaultdict(<class 'list'>, {'beta2': ['Xentro_hmgcr-like'], 'alpha2': ['Galgal_ENSGALG00000014948', 'Lepocu_ENSLOCG00000005433']})), ('OG_1007', defaultdict(<class 'list'>, {'beta2': ['Xentro_camk4', 'Lepocu_ENSLOCG00000001661', 'Xentro_LOC100496389', 'Galgal_ENSGALG00000015203'], 'alpha2': ['Galgal_ENSGALG00000000244', 'Lepocu_ENSLOCG00000010354']})

In [147]:
clgNb={og:len(ct) for og,ct in clgCts.items()}
clgPck={og:ct.most_common(1)[0][0] for og,ct in clgCts.items()}

In [148]:
hsaRef={}
for og in rTrees:
    hgns=[]
    for l in rTrees[og]:
        if l.species=='Homsap':
            hg=l.name.split('_')[1]
            if hg in HsaN:
                #hgns.append('{}:{}'.format(hg,HsaN[hg]))
                hgns.append(HsaN[hg])

    hsaRef[og]=','.join(hgns)

In [149]:
for fs,ct in Counter([frozenset(evn) for fid,evn in famEvents.items()]).items():
    print(' '.join(list(fs)),ct)

 2456
alpha2 beta2 2443
alpha1 beta1 2503
beta1 alpha1 alpha2 beta2 1856


In [150]:
evO=['alpha1','alpha2','beta1','beta2','ab1','ab2','12']
evN0=['Nd_a1','Nd_a2','Nd_b1','Nd_b2','Nd_ab1','Nd_ab2','Nd_12']
oriresflat=[]
for fid in oriRes:
    oriresflat.append([fid]+[oriRes[fid].get(ev,nan)for ev in evO])
df = pd.DataFrame(oriresflat, columns =['fid']+evN0).set_index('fid')    

In [151]:
df["CLG"] = pd.Series(clgPck)
df["NbCLG"] = pd.Series(clgNb)
df["HsaNms"] = pd.Series(hsaRef)
df["N12_Dup"] = pd.Series(nd12)

In [152]:
df

,Nd_a1,Nd_a2,Nd_b1,Nd_b2,Nd_ab1,Nd_ab2,Nd_12,CLG,NbCLG,HsaNms,N12_Dup
fid,,,,,,,,,,,
OG_1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLGB,1,POLE,NA
OG_1001,NaN,NaN,NaN,Osteichthyes,NaN,NaN,NaN,CLGC,1,MAU2,N
OG_1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLGC,1,GTDC1,NA
OG_1003.1,NaN,Osteichthyes,NaN,NaN,NaN,NaN,NaN,CLGC,1,NaN,N
OG_1004,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,CLGC,1,RPL4,N
...,...,...,...,...,...,...,...,...,...,...,...
OG_996,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,CLGA,1,HECTD1,N
OG_997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLGC,1,MECR,NA
OG_998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CLGJ,1,C1orf123,NA


In [153]:
link={}
for og in rrTrees:
    t=rrTrees[og]
    link[og]='http://202.182.123.7/trees/{}.png'.format(og)
    #t.render("png_trees/{}.png".format(og), w=180,units="mm")

### Check cyclostomes further

In [68]:
dupCyc={}
dupLfCyc={}
for fid in rrTrees:
    dec,acd=checkCyclo(rrTrees[fid],spT)
    dupCyc[fid]=dec
    dupLfCyc[fid]=acd

In [66]:
print(list(dupLfCyc.items())[0:19])

[('OG_10006', []), ('OG_1000', [['Eptbur_Eptbu0039275', 'Eptbur_Eptbu0002172', 'Parata_EA11680', 'Eptbur_Eptbu0040463', 'Parata_EA10592']]), ('OG_1001', [['Eptbur_Eptbu0036718', 'Eptbur_Eptbu0038182', 'Parata_EA09890', 'Eptbur_Eptbu0038084', 'Eptbur_Eptbu0042124', 'Parata_EA12192'], ['Eptbur_Eptbu0038084', 'Eptbur_Eptbu0042124', 'Parata_EA12192']]), ('OG_1002', []), ('OG_1003.1', []), ('OG_1003.2', []), ('OG_1004', []), ('OG_1005', []), ('OG_1006', []), ('OG_1007', []), ('OG_1009', []), ('OG_1011', []), ('OG_1012', []), ('OG_1013', []), ('OG_1014', []), ('OG_1016', [['Eptbur_Eptbu0005258', 'Parata_EA73540', 'Letjap_g13385', 'Petmar_LOC116946835', 'Eptbur_Eptbu0008385', 'Parata_EA16224']]), ('OG_1017', []), ('OG_1018', []), ('OG_1019', [['Petmar_LOC116950297', 'Letjap_g15184', 'Letjap_g15183', 'Letjap_g16626', 'Petmar_LOC116950298', 'Petmar_LOC116939115', 'Letjap_g5453', 'Letrei_Hic_chr_27_173'], ['Eptbur_Eptbu0030203', 'Parata_EA62128', 'Eptbur_Eptbu0030205', 'Parata_EA62614'], ['Petma

In [155]:
dupCycS={fid:','.join(map(str,l)) for fid,l in dupCyc.items() if not l==[]}

In [156]:
df["link"] = pd.Series(link)
df['hasCyclo']=pd.Series(hasCyclo)
df['CycloDup']=pd.Series(dupCycS)
cols = df.columns.tolist()
recols=[ 'CLG', 'NbCLG', 'N12_Dup','hasCyclo', 'CycloDup' , 'Nd_a1', 'Nd_a2', 'Nd_b1', 'Nd_b2', 'Nd_ab1', 'Nd_ab2', 'Nd_12','HsaNms','link' ]
#recols=[ 'CLG', 'NbCLG', 'N12_Dup','hasCyclo', 'CycloDup' , 'Nd_a1', 'Nd_a2', 'Nd_b1', 'Nd_b2', 'Nd_ab1', 'Nd_ab2', 'Nd_12','HsaNms' ]

df=df[recols]

In [157]:
df[df['NbCLG'] == 1]

,CLG,NbCLG,N12_Dup,hasCyclo,CycloDup,Nd_a1,Nd_a2,Nd_b1,Nd_b2,Nd_ab1,Nd_ab2,Nd_12,HsaNms,link
fid,,,,,,,,,,,,,,
OG_1000,CLGB,1,NA,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLE,http://202.182.123.7/trees/OG_1000.png
OG_1001,CLGC,1,N,10,2,NaN,NaN,NaN,Osteichthyes,NaN,NaN,NaN,MAU2,http://202.182.123.7/trees/OG_1001.png
OG_1002,CLGC,1,NA,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTDC1,http://202.182.123.7/trees/OG_1002.png
OG_1003.1,CLGC,1,N,7,0,NaN,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,http://202.182.123.7/trees/OG_1003.1.png
OG_1004,CLGC,1,N,5,"0,0",Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,RPL4,http://202.182.123.7/trees/OG_1004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OG_996,CLGA,1,N,5,0,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,HECTD1,http://202.182.123.7/trees/OG_996.png
OG_997,CLGC,1,NA,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MECR,http://202.182.123.7/trees/OG_997.png
OG_998,CLGJ,1,NA,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C1orf123,http://202.182.123.7/trees/OG_998.png


In [ ]:
print(rrTrees['OG_1000'])

In [111]:
df.to_csv('list_fams_unfilt_0220.tsv',sep="\t")

In [ ]:
# some filtereing :
#exclCLG=['CLGA', 'CLGB','CLGG','CLGH']
exclCLG=[]
print(Counter(df[df.CLG.isin(exclCLG) & ~df.Nd_12.isna()]['Nd_12']))
dfft=df[df['NbCLG'] == 1 & ~df.CLG.isin(exclCLG)]
dfft.to_csv('list_fams_filt.tsv',sep="\t")
#df[df.CLG.isin(exclCLG)]



In [171]:
#df_filt=df[(df['hasCyclo']>0) & (df['NbCLG']==1) & (df['N12_Dup']=='Y') & ~df.Nd_12.isna() & (df['Nd_12']=='Gnathostomata')]
#df_filt=df[(df['hasCyclo']>0) & (df['NbCLG']==1) & ~df.Nd_12.isna()]
#df_filt=df[(df['NbCLG']==1) & ~df.Nd_12.isna()]
df_filt=df[(df['hasCyclo']>0) & (df['NbCLG']==1)]
#df_filt=df[(df['hasCyclo']>0) & ~df.Nd_12.isna()]
#df_filt=df[~df.Nd_12.isna()]

In [172]:
df_filt

,CLG,NbCLG,N12_Dup,hasCyclo,CycloDup,Nd_a1,Nd_a2,Nd_b1,Nd_b2,Nd_ab1,Nd_ab2,Nd_12,HsaNms,link
fid,,,,,,,,,,,,,,
OG_1000,CLGB,1,NA,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POLE,http://202.182.123.7/trees/OG_1000.png
OG_1001,CLGC,1,N,10,2,NaN,NaN,NaN,Osteichthyes,NaN,NaN,NaN,MAU2,http://202.182.123.7/trees/OG_1001.png
OG_1002,CLGC,1,NA,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GTDC1,http://202.182.123.7/trees/OG_1002.png
OG_1003.1,CLGC,1,N,7,0,NaN,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,http://202.182.123.7/trees/OG_1003.1.png
OG_1004,CLGC,1,N,5,"0,0",Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,RPL4,http://202.182.123.7/trees/OG_1004.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OG_996,CLGA,1,N,5,0,Osteichthyes,NaN,NaN,NaN,NaN,NaN,NaN,HECTD1,http://202.182.123.7/trees/OG_996.png
OG_997,CLGC,1,NA,10,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MECR,http://202.182.123.7/trees/OG_997.png
OG_998,CLGJ,1,NA,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C1orf123,http://202.182.123.7/trees/OG_998.png


In [173]:
allcts=[Counter(df_filt[ev]) for  ev in evN0]
allcld=set(sum([list(ct.keys()) for ct in allcts],[]))

In [174]:
clades=[ 'Deuterostomia', 'Chordata', 'Vertebrata', 'Gnathostomata', 'Osteichthyes', 
        'Actinopterygii','Holostei', 'Sarcopterygia',   'Tetrapoda','Amniotes']

In [175]:
ctsmrg=[]
for cl in clades:
    ctsmrg.append([cl]+[ct.get(cl,0) for ct in allcts])

In [176]:
cts = pd.DataFrame(ctsmrg, columns =['clade']+evO)

In [177]:
cts

,clade,alpha1,alpha2,beta1,beta2,ab1,ab2,12
0,Deuterostomia,4,3,3,1,2,1,19
1,Chordata,5,10,5,5,6,8,24
2,Vertebrata,101,88,36,59,169,175,962
3,Gnathostomata,297,256,47,121,324,301,516
4,Osteichthyes,2237,2198,617,588,25,270,98
5,Actinopterygii,0,0,2,1,0,0,0
6,Holostei,0,0,0,0,0,0,0
7,Sarcopterygia,1,1,0,0,0,0,0
8,Tetrapoda,94,74,28,33,3,9,33
9,Amniotes,0,0,1,0,0,0,0


## construct list of duplicates gene per species for functional analyses

simpler version just looking at genes excluding tandem genes

In [27]:
def listDup(sp,rrTrees=rrTrees):
    j=0
    #with open('Xentro_ortho')
    ctPairs=[]
    outL=[]
    for og,t in rrTrees.items():
        #print(og)
        famsp={}
        spset=set([l.species for l in t])
        #if not 'Braflo' in spset: continue
        splf1=[l.name.split('_',1)[1] for l in t.search_nodes(species=sp)]
        #splf2=[l.name.split('_',1)[1] for l in t.search_nodes(species='Petmar')]
        splf=[l.name for l in t.search_nodes(species=sp)]
        #if len(splf)>1: continue
        #print(og,splf)
        agn=age(t.get_common_ancestor(splf),spT) if len(splf)>1 else sp
        #print(len(splf),agn)
        for g in splf1:
            outL.append((og,g,len(splf),agn))
        #   out.write('')
        #print(og,splf1,splf2)
        #ctPairs.append('{}_{}'.format(len(splf1),len(splf2)))

        j+=1
        #if j==200: break     
    return(outL)

In [ ]:
print(spT)

In [92]:
rrTrees['OG_1018'].get_common_ancestor(['Petmar_LOC116938252', 'Petmar_GPD1L'])

Tree node '84' (0x7f8600c3331)

In [24]:
Blan_ofam=listDup('Bralan')

In [97]:
Pata_ofam=listDup('Parata')

In [ ]:
Pata_ofam

In [26]:
with open('Lepocu_ohnofams.txt','w') as out:
    out.write("fid\tgid\tdups\tage\n")
    for l in listDup('Lepocu'):
        out.write('\t'.join(map(str,l))+'\n')

In [101]:
with open('Xentro_ohnofams.txt','w') as out:
    out.write("fid\tgid\tdups\tage\n")
    for l in listDup('Xentro'):
        out.write('\t'.join(map(str,l))+'\n')

In [28]:
with open('Letjap_ohnofams.txt','w') as out:
    out.write("fid\tgid\tdups\tage\n")
    for l in listDup('Letjap'):
        out.write('\t'.join(map(str,l))+'\n')

In [52]:
print(rrTrees['OG_1005'])


         /-Ptyfla_pfl_40v0_9_20150316_1g25190
      /-|
     |   \-Sackow_Sakowv30031522m
   /-|
  |  |   /-Acapla_LOC110979182
  |   \-|
  |      \-Strpur_LOC579449
--|
  |      /-Bralan_BL04271
  |   /-|
  |  |   \-Bralan_BL95282
   \-|
     |   /-Brabel_102460F
      \-|
        |   /-Bralan_BL06437
         \-|
            \-Braflo_BF07828


In [61]:
nV=[l.name for l in spT.search_nodes(name='Vertebrata')[0]]

In [62]:
print(nV)

['Letjap', 'Letrei', 'Petmar', 'Parata', 'Eptbur', 'Calmil', 'Chipun', 'Lepocu', 'Amical', 'Danrer', 'Orylat', 'Latcha', 'Xentro', 'Galgal', 'Homsap', 'Musmus']


In [99]:
def redChr(pos,cutoff=1000000):
    if len(pos)<2: return pos
    spo=sorted(pos,key=lambda x:(x[0],x[1]))
    rep=[spo[0]]
    #print(spo)
    for i,p in enumerate(spo[1:]):
        if p[0]!=rep[-1][0]:
            rep.append(p)
        else:
            if abs(rep[-1][1]-p[1])>cutoff:
                rep.append(p)
            else:
                continue
    #print('>>',rep)
    return rep
    

In [115]:
i=0
sp='Lepocu'
gfs=[]
gfa=[]
sela={'Deuterostomia','Vertebrata','Chordata'}
Pos=spPos[sp]
for og,t in rrTrees.items():
    i+=1
    fag=age(t,spT)
    nbV=sum([1 for l in t if l.species in nV])
    if nbV==0: continue
    splf=[l.name for l in t.search_nodes(species=sp)]
    agn=age(t.get_common_ancestor(splf),spT) if len(splf)>1 else sp
    splfp=[Pos[n.split('_',1)[1]] for n in splf if n.split('_',1)[1] in Pos]
    splfd=redChr(splfp)
    spflc=set([c[0] for c in splfp])
    #print(og,fag,spflc,agn)
    if fag in sela:
        gfs.append(len(splfd))
        gfa.append(len(splf))
    #else:
        #print(fag)
    #if i==30: break


In [116]:
print(Counter(gfs))
#print(Counter(gfa))


Counter({1: 5748, 2: 1718, 0: 1253, 3: 647, 4: 204, 5: 61, 6: 24, 8: 9, 7: 9, 17: 6, 9: 5, 12: 3, 13: 3, 10: 2, 21: 2, 20: 2, 11: 2, 14: 2, 24: 1})


In [109]:
print(spPos.keys())

dict_keys(['Lepocu', 'Galgal', 'Xentro', 'Braflo'])


In [36]:
from ete3 import Phyloxml

In [37]:
project = Phyloxml()
project.build_from_file("/Users/fmarletaz/Dropbox (UCL)/Drop/embaezzle/OG_217_reconciliated.xml")

In [45]:
project.get_phylogeny()

[]

In [71]:
from ReconciledTree import getEventsSummary

ImportError: cannot import name 'getEventsSummary' from 'ReconciledTree' (/Users/fmarletaz/Dropbox (UCL)/Genomes/Hagfish/Gene_families/ReconciledTree.py)

In [43]:
for tree in project.get_phylogeny():
    print(tree)
    # you can even use rendering options
    tree.show()
    # PhyloXML features are stored in the phyloxml_clade attribute
    for node in tree:
        print( "Node name:", node.name)
    

In [ ]:
listDup('Bralan')

In [60]:
def listPairPos(sp,Pos,rrTrees=rrTrees):
    j=0
    outL=[]
    for og,t in rrTrees.items():
        #print(og)
        famsp={}
        spset=set([l.species for l in t])
        #print(spset)
        #if not 'Braflo' in spset: continue
        splf=[l.name for l in t.search_nodes(species=sp)]
        if len(splf)<2: continue
        #print(og,splf,list(itertools.combinations(splf, 2)))
        for g1,g2 in itertools.combinations(splf, 2):
            pan=t.get_common_ancestor([g1,g2])
            agn=age(pan,spT)
            dps=pan.D
            g1n,g2n=[l.split('_',1)[1] for l in (g1,g2)]
            e=[og,agn,g1n,Pos[g1n][0],Pos[g1n][1],Pos[g1n][2],g2n,Pos[g2n][0],Pos[g2n][1],Pos[g2n][2],dps]
            outL.append(e)
        #splf2=[l.name.split('_',1)[1] for l in t.search_nodes(species='Petmar')]
        #splf=[l.name for l in t.search_nodes(species=sp)]
        #if len(splf)>1: continue
        #agn=age(t.get_common_ancestor(splf),spT) if len(splf)>1 else sp
        #print(len(splf),agn)
        #for g in splf1:
        #    outL.append((og,g,len(splf),agn))
        #   out.write('')
        #print(og,splf1,splf2)
        #ctPairs.append('{}_{}'.format(len(splf1),len(splf2)))
    
        j+=1
        #if j==5: break
    return outL

In [61]:
with open('Petmar_agepairs.txt','w') as out:
    out.write("fid\tage\tg1\tg1_chr\tg1_start\tg1_end\tg2\tg2_chr\tg2_start\tg2_end\n")
    for oL in listPairPos('Petmar',Pma):
        out.write('\t'.join(map(str,oL))+'\n')

In [ ]:
print(rrTrees['OG_1033'])

## Coordinates of lamprey duplications

In [37]:
def Cyclo12(t,spT):
    spCycl=[l.name for l in spT.search_nodes(name="Cyclostomata")[0].get_leaves()]
    for l in t:
        cla='Cyclostomata' if l.species in spCycl else 'Other'
        l.add_features(clade=cla)
    dec=[]
    acd=[]
    
    cnd=[st for st in t.get_monophyletic(values=['Cyclostomata'], target_attr="clade")]
    return cnd

In [ ]:
print(pmaPairs[frozenset(('scaf_00055', 'scaf_00055'))])
print(pmaPairs[frozenset(('scaf_00014', 'scaf_00055'))])
print(list(itertools.product(['OG_1173', 'OG_16312', 'OG_17608'],['OG_2400', 'OG_361', 'OG_4328', 'OG_5513', 'OG_7398'])))

In [46]:

#pmaChrm=defaultdict(list)
pmaPair=defaultdict(list)
cnaAll={}
i=0
cyclostomes=set(['Parata','Eptbur','Letrei','Petmar','Letjap'])
#spCycl=[l.name for l in spT.search_nodes(name="Cyclostomata")[0].get_leaves()]
pmaPairs=defaultdict(list)
pmaPairsZ=defaultdict(list)
pmaPairsP=defaultdict(list)

for fid,ftr in rrTrees.items():
    #print(i,fid)
    spset=set([l.species for l in ftr])
    if not 'Braflo' in spset: continue
    #clgct,anDup=checkDup(ftr,spPos)
    #dec,acd=checkCyclo(ftr,spT)
    #print(dec,acd)
    #if anDup.keys()
    #cmn=Cyclo12(ftr,spT)
    #print(cmn)
    #clgCts[fid]=clgct
    #oridup=oriDup(ftr,anDup,spT)
    #ori12=oriDup12(ftr,anDup,spT)
    for l in ftr:
        cla='Cyclostomata' if l.species in cyclostomes else 'Other'
        l.add_features(clade=cla)
    dec,acd=[],[]
    zDups=[st for st in ftr.get_monophyletic(values=['Cyclostomata'], target_attr="clade")]
    if len(zDups)>1:
        pma_pos,pma_gen=[],[]
        for st in zDups[0:2]:
            #pma_gen.append([l.name.split('_',1)[-1] for l in st.search_nodes(species="Petmar")])
            pma_pos.append(list(set([Pma[l.name.split('_',1)[-1]][0] for l in st.search_nodes(species="Petmar")])))
        for p in itertools.product(*pma_pos):
            pmaPairs[frozenset(p)].append(fid)
            pmaPairsZ[frozenset(p)].append(fid)
    for st in zDups:
        dupnds=[sst for sst in st.search_nodes(D='Y') if not len(set([l.species for l in sst.get_leaves()]))==1]
        for nd in dupnds:
            pma_pos=[]
            for ndd in nd.children:
                pma_pos.append([Pma[l.name.split('_',1)[-1]][0] for l in ndd.search_nodes(species="Petmar")])
            for p in itertools.product(*pma_pos):
                pmaPairs[frozenset(p)].append(fid)    
                pmaPairsP[frozenset(p)].append(fid)
   
       

    i+=1
    if i==100: break


To try to check whether pairs are specifically enriches in some chromosomes, let's try to obtain pairs of paralogue of different phylogenetic age, first for lamrpey

In [ ]:
for fid,ftr in rrTrees.items():
    

In [41]:
print(pma_pos)

[[], ['chrm62']]


## Retention rates

In [121]:
def calcRet(species,Pos,trees=rrTrees,spPos=spPos):
    j=0
    allclgs=set()
    retChr=defaultdict(list)
    for og,t in trees.items():
        #print(i,fid)
        spset=set([l.species for l in t])
        if not 'Braflo' in spset: continue
        clgs,anDup=checkDup(t,spPos)
        if not len(clgs)==1: continue
        #print(og,clgs,list(clgs))
        scafs=set()
        for hn in t.search_nodes(species=species):
            pg=hn.name.split('_',1)[1]
            if not pg in Pos: continue
            scaf,start,end=Pos[pg]
            scafs.add(scaf)

        #allclgs.add(list(clgs)[0])
        #rclg.add(list(clgs)[0])
                #print(list(clgs)[0])
        #print(rclg)
        allclgs.add(list(clgs)[0])
        for scaf in scafs:
            retChr[scaf].append(list(clgs)[0])
            #print(hn.name,Pat[pg])
        j+=1
        #if j==10: break
    #print(retChr)
    byCLG = pd.DataFrame(columns=list(allclgs))
    for chrm in retChr:
        clgCt=Counter(retChr[chrm])
        #print(chrm,clgCt)
        byCLG.loc[chrm]=pd.Series(clgCt)
    return byCLG

In [122]:
Pata_ct=calcRet('Parata',Pat)
Pata_ret=Pata_ct/Pata_ct.sum()

In [123]:
Pmar_ct=calcRet('Petmar',Pma)
Pmar_ret=Pmar_ct/Pmar_ct.sum()

In [124]:
Pata_ret.to_csv('Pata_retention.tsv',sep='\t')
Pmar_ret.to_csv('Pmar_retention.tsv',sep='\t')

In [ ]:
byCLG = pd.DataFrame(columns=list(allclgs))
for chrm in retChr:
    clgCt=Counter(retChr[chrm])
    #print(chrm,clgCt)
    byCLG.loc[chrm]=pd.Series(clgCt)

In [127]:
pmaChGct=defaultdict(int)
for g,p in Pma.items():
    pmaChGct[p[0]]+=1
selPmaChr={g:n for g,n in sorted(pmaChGct.items(),key=lambda x:x[1],reverse=True)[0:100]}

In [134]:
def writeDupScafMat(pmaPairs,filename):
    with open(filename,'w') as out:
        out.write("\t{}\n".format('\t'.join(selPmaChr)))
        for sc1 in selPmaChr:
            onb=[len(pmaPairs[frozenset((sc1,sc2))]) for sc2 in selPmaChr]
            out.write("{}\t{}\n".format(sc1,'\t'.join(map(str,onb))))
            
            #print(sc1,sc2,len(pmaPairs[frozenset((sc1,sc2))]))
                #out.write("{}\t{}\t{}\n".format(sc1,sc2,len(ogs)))


In [135]:
writeDupScafMat(pmaPairs,'nb_dup_all.txt')
writeDupScafMat(pmaPairsZ,'nb_dup_zero.txt')
writeDupScafMat(pmaPairsP,'nb_dup_post.txt')

# Prepare calibrations

In [27]:
def calib(events,calibs,tree):
    taxCal=[]
    ancestors=set()
    for ev in events:
        if ev.etype == "S":
            #print 'ORTHOLOGY RELATIONSHIP:', ','.join(ev.inparalogs), "<====>", ','.join(ev.orthologs)
            #inpTx,ortTx=[n.split('_')[0] for n in ev.inparalogs],[n.split('_')[0] for n in ev.orthologs] 
            for tx1,tx2 in list(itertools.product(ev.inparalogs,ev.orthologs)):
                anc12=tree.get_common_ancestor([tx1,tx2])
                if not anc12 in ancestors:
                    sp1,sp2=tx1.split('_')[0],tx2.split('_')[0]
                    #print tx1,tx2,sp1,sp2
                    if (sp1,sp2) in calibs:
                        maxage,minage=calibs[(sp1,sp2)]
                        taxCal.append((tx1,tx2,maxage,minage))
                        ancestors.add(anc12)
                        break
                else:
                    print("we already have this one! ")
    return taxCal


In [28]:
from tqdm import tqdm_notebook

In [29]:
import itertools

Load calibrations

In [30]:
calibs={}
for line in open('calibrations2.txt'):
    sps1,sps2,maxage,minage=line.strip().split('\t')
    sps1,sps2=sps1.split(','),sps2.split(',')
    for pair in itertools.product(sps1,sps2):
        calibs[pair]=maxage,minage
        calibs[pair[::-1] ]=maxage,minage


In [33]:
ct=0
selTrees={}
for og,t in rrTrees.items():
    if not og in oriRes: continue
    if len(oriRes[og])>1:
        selTrees[og]=t
    #if not 'Deuterostomia' in oriRes[og].values() and len(oriRes[og])>1:
        #print(oriRes[og])

print(len(selTrees))

2250


In [34]:
spTree=PhyloTree(open('deut2.tre').read(), format=1)

In [ ]:
#selTrees['OG_1000'].reconcile(spT)
print(selTrees['OG_1000'].write(format=9))
t=PhyloTree(selTrees['OG_1000'].write(format=9))
t.set_species_naming_function(lambda node: node.name.split("_")[0] )

rctree,events=t.reconcile(spTree)

In [35]:
for og in tqdm_notebook(selTrees):
    t=PhyloTree(selTrees[og].write(format=9))
    t.set_species_naming_function(lambda node: node.name.split("_")[0] )
    try:
        rctree,events=t.reconcile(spTree)
        tcal=calib(events,calibs,t)
        with open('calibrations/{0}.calib'.format(og),'w') as out:
            out.write("{0}\n".format(len(tcal)))
            for cl in tcal:
                out.write("{0}\n".format('\t'.join(cl)))
    except:
        print("problem with",og)


                

<ipython-input-35-c0f0d61b38d4>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for og in tqdm_notebook(selTrees):


In [95]:
for og in tqdm_notebook(selTrees):
    PhyloTree(selTrees[og].write(outfile='tree4dating/{0}.tre'.format(og),format=9))

/Users/fmarletaz/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


# Results of molecular dating 

To read and interpret the chronograms, we have to activate compute the node age by summing up all the branch length (=time) to the closest lead for each node. Figtree seems to do that automatically, but we need to do it ourselves here!

In [66]:
chronos={}
chronoGenes={}
for l in open('Chronograms_0221_nc.trees'):
    o,ts=l.strip().split(':',1) 
    og=o.replace('_nc_sample.chronogram','')
    g2s=defaultdict(list)
    t=PhyloTree(ts,format=1,sp_naming_function=lambda x:x.split("_")[0])
    for l in t:
        sp,gid=l.name.split('_',1)
        gid=gid.split('.')[0]
        l.add_features(species=sp,gid=gid)
        g2s[sp].append(gid)
    chronoGenes[og]=g2s
    for nd in t.traverse("postorder"):
        if not nd.is_leaf():
            oneChild=nd.get_closest_leaf(topology_only=False, is_leaf_fn=None)
            nd.add_features(age=oneChild[1])
    chronos[og]=t

           

In [63]:
print(len(chronoGenes))

2245


In [ ]:
print(rrTrees['OG_1000'].get_ascii(attributes=["species", "clade","D","name"], show_internal=True))

In [64]:
def extDup(anDup):
    if 'alpha1'  in anDup and 'beta1' in anDup:
        anDup['ab1']=anDup['alpha1']+anDup['beta1']
    if 'alpha2'  in anDup and 'beta2' in anDup:
        anDup['ab2']=anDup['alpha2']+anDup['beta2']
    if all(item in [nd[-1] for nd in anDup] for item in ['1','2']):
        anDup['12']=list(set(chain(*[anDup[nd] for nd in anDup])))
    return anDup

In [69]:
ageEvn,CLGs={},{}
AnnTet={}
n=0
for og,t in chronos.items():
    #print(og)
    tax=[l for l in t if l.species=='Braflo']        
    if len(tax)==0: continue
    clg,anDup=checkDup(t,spPos)
    CLGs[og]=clg
    datRes={}
    #print(clg)
    anDup=extDup(anDup)
    #dupnds=t.split_by_dups()
    #print(og,clg,anDup)
    #if len(clg)>1: continue
    tr=rrTrees[og]
    
    for cdln in dupLfCyc[og]:
        txl=set([t.split('_')[0] for t in cdln])
        cdns=t.get_common_ancestor(cdln)
        cld=spT.get_common_ancestor(txl)
        datRes['Cyc_{}'.format(cld.name)]=cdns.age
        #datRes['Cyc_{}'.format('.'.join(txl))]=cdns.age
        #print('>',og,cdns.age,txl)

        #print('>',og,cdns.age,txl) 
    #for nd in dupnds:
    #    print(nd)
    for evdp in anDup:
        evgn=anDup[evdp]
        evsp=set([l.split('_')[0] for l in evgn])
        evnd=t.get_common_ancestor(anDup[evdp])
        dns=tr.get_common_ancestor(anDup[evdp])
        #print(og,evdp,evsp,evgn,dns.D,evnd.age)
        #evnd.img_style["size"] = 50
        #evnd.img_style["fgcolor"] = col[evdp]
        if evdp in ['ab1','ab2','12'] and len(evsp)>1 and dns.D=='Y':
            #print(og,evdp,evsp,evgn,evnd.age)
            datRes[evdp]=evnd.age
    ageEvn[og]=datRes
    n+=1
    #if n==10: break

In [193]:
list(datRes.items())[0:5]

[('Cyc_Letjap.Letrei.Petmar.Eptbur.Parata', 481.7593999999999),
 ('Cyc_Letjap.Eptbur.Petmar.Parata', 466.3526),
 ('Cyc_Letjap.Letrei', 174.4184),
 ('Cyc_Eptbur.Parata', 319.957),
 ('Cyc_Petmar.Letjap.Letrei', 275.1524)]

In [70]:
n=0
cladeAge=defaultdict(list)
#iclades=set(['Deuterostomia','Chordata','Vertebrata','Gnathostomata','Cyclostomata','Osteichthyes','Hyperoartia'])
for og,t in chronos.items():
    #print(og)
    tr=rrTrees[og]
    for n in tr.search_nodes(D='N'):
        if n.is_leaf(): continue
        lfs=[lf.name for lf in n]
        dn=t.get_common_ancestor(lfs)
        if not age(n,spT)=='':
            cladeAge[age(n,spT)].append((og,dn.age))
            #print(n.name,age(n),dn.age)
    

In [71]:
def checkGnatho(t,spT):
    spCycl=[l.name for l in spT.search_nodes(name="Gnathostomata")[0].get_leaves()]
    print(spCycl)
    for l in t:
        cla='Gnathostomata' if l.species in spCycl else 'Other'
        l.add_features(clade=cla)
    dec=[]
    acd=[]
    for st in t.get_monophyletic(values=['Gnathostomata'], target_attr="clade"):
        #dupnds=[age(sst) for sst in st.search_nodes(D='Y') if not len(set([l.species for l in sst.get_leaves()]))==1]
        dupnds=[sst for sst in st.search_nodes(D='Y') if not len(set([l.species for l in sst.get_leaves()]))==1]
        #print(dupnds)
        #print(st.get_ascii(attributes=["gid","species", "clade","D"], show_internal=True))
        #ntrees, ndups, sptrees =  st.get_speciation_trees()
        dec.append(len(dupnds))
        for nd in dupnds:
            acd.append([l.name for l in nd.get_leaves()])
        #print(ntrees, ndups)
        #for std in sptrees:
        #    print(std.get_ascii(attributes=["species", "clade"], show_internal=False))
    return dec,acd

In [95]:
print(rrTrees['OG_10507'].get_ascii(attributes=["name", "D"], show_internal=True))


                   /-Sackow_Sakowv30045477m, N
             /n21, N
            |      \-Ptyfla_pfl_40v0_9_20150316_1g18012, N
       /n30, N
      |     |            /-Strpur_LOC115919062, N
      |     |      /n24, Y
      |      \n27, N     \-Strpur_LOC579973, N
      |           |
      |            \-Acapla_LOC110977617, N
      |
      |                  /-Braflo_BF14743, N
      |            /n69, N
-n153, N     /n72, N     \-Bralan_BL12789, N
      |     |     |
      |     |      \-Brabel_230050R, N
      |     |
      |     |                              /-Xentro_alkbh1, N
      |     |                        /n39, N
      |     |                       |     |      /-Galgal_ENSGALG00000010485, N
      |     |                       |      \n36, N
      |     |                  /n42, N          |      /-Musmus_ENSMUSG00000079036, N
      |     |                 |     |            \n33, N
       \n75, N                |     |                  \-Homsap_ENSG00000100601, N
       

In [72]:
cladeDup=defaultdict(int)
clade='Gnathostomata'
clSp=[l.name for l in spT.search_nodes(name=clade)[0].get_leaves()]
gnathDup={}
gnathAge=[]
j=0
for og,tc in chronos.items():
    try:
        if not og in rrTrees: continue
        tr=rrTrees[og]
        clg,anDup=checkDup(tc,spPos)
        gnLf=[l.name for l in tr if l.species in clSp]
        if len(gnLf)==0: continue
        clAnc=tr.get_common_ancestor(gnLf)
        for dn in clAnc.search_nodes(D='Y'):
            nsp=len(set([l.species for l in dn.get_leaves()]))
            if nsp==1: continue
            lfs=[lf.name for lf in dn]
            
            gnathDup[og]=lfs
            dnc=tc.get_common_ancestor(lfs)
            gnathAge.append((og,'InGnathostomes',','.join(clg),dnc.age))
            #print(age(dn,spT),nsp,dnc.age)
            #print(dn.get_ascii(attributes=["name", "D"], show_internal=True))
    except Exception as ex:
        print(og)
        print(ex)
    
    j+=1
    #if j==50: break
print(j)
    


2245


In [75]:
with open('age_gnathoDup_v5.txt','w') as out:
    for r in gnathAge:
        out.write('\t'.join(map(str,r))+'\n')

In [ ]:
print(list(ageEvn.items())[0:4])
print(CLGs['OG_1000'])

print(len(ageEvn))

In [74]:
with open('ages_clades_v5.txt','w')as oat:
    for sp in cladeAge:
        for og,age in cladeAge[sp]:
            oat.write("{}\t{}\t{}\n".format(og,age,sp))
    

In [73]:
with open('ages_v5.tsv','w') as out:
    for og in ageEvn:
        clgs=CLGs[og]
        dres=ageEvn[og]
        if dres=={}: continue
        for ev in dres:
            out.write("{}\t{}\t{}\t{}\n".format(og,ev,','.join(clgs),dres[ev]))

In [130]:
checkCyclo(rrTrees['OG_1000'],spT)

([0, 0], [])

In [129]:
print(rrTrees['OG_1000'].get_ascii(attributes=["D","name","Clade"], show_internal=True))


                            /-N, Strpur_LOC586037
                     /Y, n138
              /N, n141      \-N, Strpur_LOC115918720
             |      |
       /N, n147      \-N, Acapla_LOC110973119
      |      |
      |      |       /-N, Sackow_Sakowv30009015m
      |       \N, n144
      |              \-N, Ptyfla_pfl_40v0_9_20150316_1g9723
      |
      |                            /-N, Bralan_BL10436
      |                     /N, n120
      |              /N, n123      \-N, Braflo_BF11510
      |             |      |
      |             |       \-N, Brabel_170030F
      |       /Y, n132
      |      |      |              /-N, Bralan_BL77595
-N, n297     |      |       /N, n126
      |      |       \N, n129      \-N, Braflo_BF11238
      |      |             |
      |      |              \-N, Brabel_117710F
      |      |
      |      |                                                 /-N, Musmus_ENSMUSG00000027247
      |      |                                           /N, n3

# Checking some genes

In [ ]:
Hsa_id={}

In [66]:
Hsa2OG={}
for og,t in rrTrees.items():
    for n in t.search_nodes(species="Homsap"):
        #print(n.name.split('_')[1])
        Hsa2OG[n.name.split('_')[1]]=og   

In [126]:
Hsa2OG['ENSG00000203883']

'OG_10995'

In [123]:
print(rrTrees['OG_3140'])


         /-Strpur_LOC593520
      /-|
     |   \-Acapla_LOC110978340
   /-|
  |  |   /-Ptyfla_pfl_40v0_9_20150316_1g912
  |   \-|
  |      \-Sackow_Sakowv30020282m
  |
  |      /-Brabel_212320R
  |   /-|
  |  |  |   /-Braflo_BF07649
  |  |   \-|
  |  |      \-Bralan_BL06291
  |  |
  |  |            /-Letjap_g9933
  |  |         /-|
  |  |      /-|   \-Letrei_Hic_chr_22_57
  |  |     |  |
--|  |     |   \-Petmar_LOC116941195
  |  |     |
  |  |     |            /-Danrer_ENSDARG00000025847
  |  |     |         /-|
  |  |     |        |   \-Lepocu_ENSLOCG00000018365
  |  |     |        |
  |  |     |      /-|         /-Musmus_ENSMUSG00000051817
  |  |     |     |  |      /-|
  |  |     |     |  |   /-|   \-Homsap_ENSG00000177732
  |  |     |     |  |  |  |
  |  |   /-|   /-|   \-|   \-Galgal_ENSGALG00000035951
  |  |  |  |  |  |     |
  |  |  |  |  |  |      \-Latcha_ENSLACG00000007378
  |  |  |  |  |  |
  |  |  |  |  |  |   /-Chipun_Chipu0009345
   \-|  |  |  |   \-|
     |  |  |  |    